# Territorial.io Vision Model Training
## Train a CNN to classify game map regions

**Instructions:**
1. Upload this notebook to [Kaggle](https://kaggle.com/notebooks)
2. Enable GPU: Settings → Accelerator → GPU T4 x2 (free)
3. Run all cells
4. Download `vision_model.pth` from the Output tab
5. Place it in `territorial_bot/models/vision_model.pth`

This notebook:
- Generates synthetic training data from Territorial.io color patterns
- Trains a CNN (TerritoryClassifierCNN) to classify map patches
- Exports the trained model weights

In [ ]:
# Install dependencies
!pip install -q torch torchvision opencv-python-headless Pillow numpy matplotlib scikit-learn

In [ ]:
import os
import random
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
from pathlib import Path

# Kaggle output directory
OUTPUT_DIR = '/kaggle/working'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

## 1. Synthetic Data Generation

Since we can't scrape the live game directly in Kaggle, we generate realistic synthetic training data
based on Territorial.io's known color palette and visual patterns.

In [ ]:
# ─────────────────────────────────────────────
# TERRITORIAL.IO COLOR PALETTE
# ─────────────────────────────────────────────

# Territory colors (RGB) - these are the actual colors used in the game
TERRITORY_COLORS = {
    'own': [
        (0, 100, 255),    # Blue (default player color)
        (0, 150, 255),
        (30, 120, 220),
        (0, 80, 200),
    ],
    'enemy': [
        (255, 50, 50),    # Red
        (220, 30, 30),
        (255, 80, 80),
        (200, 0, 0),
        (255, 140, 0),    # Orange enemy
        (180, 0, 255),    # Purple enemy
        (50, 200, 50),    # Green enemy
        (255, 220, 0),    # Yellow enemy
        (0, 220, 220),    # Cyan enemy
        (255, 100, 180),  # Pink enemy
    ],
    'neutral': [
        (180, 180, 180),  # Light gray
        (160, 160, 160),
        (200, 200, 200),
        (140, 140, 140),
        (170, 170, 170),
    ],
    'border': [
        (10, 10, 10),     # Near black
        (20, 20, 20),
        (5, 5, 5),
        (30, 30, 30),
    ],
}

CLASSES = ['own', 'enemy', 'neutral', 'border']
PATCH_SIZE = 64  # CNN input size
SAMPLES_PER_CLASS = 2000

print(f'Classes: {CLASSES}')
print(f'Patch size: {PATCH_SIZE}x{PATCH_SIZE}')
print(f'Samples per class: {SAMPLES_PER_CLASS}')

In [ ]:
def generate_territory_patch(label: str, size: int = 64) -> np.ndarray:
    """
    Generate a realistic synthetic patch for a given territory class.
    Adds noise, gradients, and texture to simulate real game visuals.
    """
    colors = TERRITORY_COLORS[label]
    base_color = random.choice(colors)
    
    # Create base patch
    patch = np.zeros((size, size, 3), dtype=np.uint8)
    
    if label == 'border':
        # Dark with slight variation
        noise = np.random.randint(0, 15, (size, size, 3), dtype=np.uint8)
        patch[:] = base_color
        patch = np.clip(patch.astype(int) + noise - 7, 0, 255).astype(np.uint8)
        
        # Add border lines
        if random.random() > 0.5:
            cv2.line(patch, (0, size//2), (size, size//2), (50, 50, 50), 2)
        if random.random() > 0.5:
            cv2.line(patch, (size//2, 0), (size//2, size), (50, 50, 50), 2)
    
    elif label == 'neutral':
        # Uniform gray with texture
        patch[:] = base_color
        noise = np.random.randint(-20, 20, (size, size, 3))
        patch = np.clip(patch.astype(int) + noise, 0, 255).astype(np.uint8)
        
        # Add subtle grid pattern
        if random.random() > 0.7:
            for i in range(0, size, 8):
                patch[i, :] = np.clip(patch[i, :].astype(int) - 10, 0, 255)
                patch[:, i] = np.clip(patch[:, i].astype(int) - 10, 0, 255)
    
    else:  # own or enemy
        # Solid color with gradient and noise
        patch[:] = base_color
        
        # Add gradient
        gradient = np.linspace(0, 30, size).astype(int)
        for i in range(size):
            patch[i] = np.clip(
                patch[i].astype(int) + gradient[i] - 15, 0, 255
            ).astype(np.uint8)
        
        # Add noise
        noise = np.random.randint(-15, 15, (size, size, 3))
        patch = np.clip(patch.astype(int) + noise, 0, 255).astype(np.uint8)
        
        # Occasionally add troop number text simulation
        if random.random() > 0.6:
            num = str(random.randint(1, 999))
            cv2.putText(
                patch, num,
                (random.randint(5, 20), random.randint(20, 50)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.4, (255, 255, 255), 1
            )
        
        # Add border edge effect
        if random.random() > 0.5:
            edge_width = random.randint(1, 3)
            patch[:edge_width, :] = np.clip(
                patch[:edge_width, :].astype(int) - 40, 0, 255
            ).astype(np.uint8)
    
    return patch


def generate_dataset(samples_per_class: int = 2000):
    """Generate the full synthetic training dataset."""
    images = []
    labels = []
    
    for label in CLASSES:
        print(f'Generating {samples_per_class} samples for class: {label}')
        for _ in range(samples_per_class):
            patch = generate_territory_patch(label, PATCH_SIZE)
            images.append(patch)
            labels.append(label)
    
    return np.array(images), np.array(labels)


# Generate dataset
print('Generating synthetic training data...')
images, labels = generate_dataset(SAMPLES_PER_CLASS)
print(f'Dataset: {images.shape}, Labels: {labels.shape}')

# Visualize samples
fig, axes = plt.subplots(4, 8, figsize=(16, 8))
for i, cls in enumerate(CLASSES):
    cls_indices = np.where(labels == cls)[0]
    for j in range(8):
        idx = cls_indices[j]
        axes[i, j].imshow(cv2.cvtColor(images[idx], cv2.COLOR_BGR2RGB))
        axes[i, j].set_title(cls, fontsize=8)
        axes[i, j].axis('off')
plt.suptitle('Synthetic Training Samples', fontsize=14)
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/training_samples.png', dpi=100)
plt.show()
print('Sample visualization saved.')

## 2. Dataset and DataLoader

In [ ]:
# Encode labels
le = LabelEncoder()
le.fit(CLASSES)
label_ids = le.transform(labels)
print(f'Label mapping: {dict(zip(le.classes_, le.transform(le.classes_)))}')

# Save label encoder
with open(f'{OUTPUT_DIR}/label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)
print('Label encoder saved.')


class TerritoryDataset(Dataset):
    """PyTorch Dataset for territory patch classification."""
    
    def __init__(self, images, labels, transform=None, augment=False):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.augment = augment
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx].copy()
        label = self.labels[idx]
        
        # Data augmentation
        if self.augment:
            img = self._augment(img)
        
        # Convert BGR to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(img_rgb)
        
        if self.transform:
            img_tensor = self.transform(pil_img)
        else:
            img_tensor = transforms.ToTensor()(pil_img)
        
        return img_tensor, torch.tensor(label, dtype=torch.long)
    
    def _augment(self, img):
        """Apply random augmentations."""
        # Random horizontal flip
        if random.random() > 0.5:
            img = cv2.flip(img, 1)
        
        # Random vertical flip
        if random.random() > 0.5:
            img = cv2.flip(img, 0)
        
        # Random rotation (0, 90, 180, 270)
        k = random.randint(0, 3)
        if k > 0:
            img = np.rot90(img, k).copy()
        
        # Random brightness
        if random.random() > 0.5:
            factor = random.uniform(0.7, 1.3)
            img = np.clip(img.astype(float) * factor, 0, 255).astype(np.uint8)
        
        # Random Gaussian noise
        if random.random() > 0.5:
            noise = np.random.normal(0, 10, img.shape).astype(np.int16)
            img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        
        return img


# Train/val split
X_train, X_val, y_train, y_val = train_test_split(
    images, label_ids, test_size=0.2, random_state=42, stratify=label_ids
)
print(f'Train: {len(X_train)}, Val: {len(X_val)}')

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Datasets and loaders
train_dataset = TerritoryDataset(X_train, y_train, transform=train_transform, augment=True)
val_dataset = TerritoryDataset(X_val, y_val, transform=val_transform, augment=False)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)

print(f'Train batches: {len(train_loader)}, Val batches: {len(val_loader)}')

## 3. CNN Model Definition

In [ ]:
class TerritoryClassifierCNN(nn.Module):
    """
    Lightweight CNN for classifying game map patches.
    Input: 64x64 RGB patch
    Output: 4-class softmax (own, enemy, neutral, border)
    
    Architecture matches vision_system.py for direct weight loading.
    """
    def __init__(self, num_classes: int = 4):
        super().__init__()
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),   # 32x32

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),   # 16x16

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),   # 8x8
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 8 * 8, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


# Initialize model
model = TerritoryClassifierCNN(num_classes=len(CLASSES)).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total parameters: {total_params:,}')
print(f'Trainable parameters: {trainable_params:,}')
print(model)

## 4. Training

In [ ]:
# Training configuration
EPOCHS = 30
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Training history
history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': []
}

best_val_acc = 0.0
best_model_path = f'{OUTPUT_DIR}/vision_model_best.pth'


def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    return total_loss / len(loader), 100.0 * correct / total


def val_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return total_loss / len(loader), 100.0 * correct / total


print(f'Starting training for {EPOCHS} epochs...')
print('=' * 60)

for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = val_epoch(model, val_loader, criterion)
    scheduler.step()
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        marker = ' ← BEST'
    else:
        marker = ''
    
    print(
        f'Epoch {epoch:3d}/{EPOCHS} | '
        f'Train Loss: {train_loss:.4f} Acc: {train_acc:.1f}% | '
        f'Val Loss: {val_loss:.4f} Acc: {val_acc:.1f}%{marker}'
    )

print('=' * 60)
print(f'Training complete! Best val accuracy: {best_val_acc:.1f}%')

## 5. Training Curves

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

epochs_range = range(1, EPOCHS + 1)

ax1.plot(epochs_range, history['train_loss'], 'b-', label='Train Loss')
ax1.plot(epochs_range, history['val_loss'], 'r-', label='Val Loss')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True)

ax2.plot(epochs_range, history['train_acc'], 'b-', label='Train Acc')
ax2.plot(epochs_range, history['val_acc'], 'r-', label='Val Acc')
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/training_curves.png', dpi=100)
plt.show()
print('Training curves saved.')

## 6. Evaluation and Confusion Matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Load best model
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(targets.numpy())

# Classification report
print('Classification Report:')
print(classification_report(
    all_targets, all_preds,
    target_names=le.classes_
))

# Confusion matrix
cm = confusion_matrix(all_targets, all_preds)
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=le.classes_, yticklabels=le.classes_
)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/confusion_matrix.png', dpi=100)
plt.show()

## 7. Export Final Model

In [ ]:
# Save final model (same as best)
final_model_path = f'{OUTPUT_DIR}/vision_model.pth'
torch.save(model.state_dict(), final_model_path)
print(f'Final model saved to: {final_model_path}')

# Verify model can be loaded
test_model = TerritoryClassifierCNN(num_classes=len(CLASSES))
test_model.load_state_dict(torch.load(final_model_path, map_location='cpu'))
test_model.eval()

# Test inference
dummy_input = torch.randn(1, 3, 64, 64)
with torch.no_grad():
    output = test_model(dummy_input)
    probs = torch.softmax(output, dim=1)
print(f'Test inference output shape: {output.shape}')
print(f'Test probabilities: {probs.numpy()}')
print('Model verification passed!')

# List all output files
print('\nOutput files:')
for f in sorted(Path(OUTPUT_DIR).iterdir()):
    size_kb = f.stat().st_size / 1024
    print(f'  {f.name}: {size_kb:.1f} KB')

print('\n✅ DONE! Download vision_model.pth and label_encoder.pkl')
print('   Place them in territorial_bot/models/')